# Algebra Komputerowa 
## Laboratorium 02 - Algorytmy Modularne 

Plan Laboratorium
1) Zastosowanie Chińskiego Twierdzenia o Resztach do rozwiązywania Układów Kongruencji
    - Metoda Lagrange'a
    - Metoda Przyrostowa Newtona
2) Szybkie potęgowanie Modularne
    - Standardowa wersja 
    - Wykorzystanie Twierdzenia Eulera
3) Liczenie wyznacznika Macierzy Całkowitoliczbowej za pomocą Chińskiego Twierdzenia o Resztach
4) Zadanie dodatkoawe - FFT

Importy

In [1]:
using Oscar
using Random
using StatsBase
using Test
using Primes

### Używając paczki `Primes` zapisujemy dużą tablicę liczb pierwszych. Przyda nam się to później.

Z powodu niejednoznaczności nazwy funkcji `primes` (taka sama funckja występuje w paczce `Oscar`), musimy ręcznie wskazać o ktorą nam chodzi. 

In [2]:
primes = Primes.primes 
PRIMES = BigInt.(primes(1,1000000))

78498-element Vector{BigInt}:
      2
      3
      5
      7
     11
     13
     17
     19
     23
     29
      ⋮
 999883
 999907
 999917
 999931
 999953
 999959
 999961
 999979
 999983

Konwersja do `BigInt` (nieograniczonego Integera) jest nieprzypadkowa. Przez całe laboratorium będziemy działać na olbrzymich liczbach całkowitych. 

Standardowo Julia wykorzystuje `Int64`.
Można też stosować `Int128`, ale okazuje się, że dla naszych celów, to wciaż dużo za mało. 

Będziemy rozważać duże układy kongruencji. Ostatecznie, będziemy chcieli mnożyć conajmniej tyle liczb pierwszych ile występuje kongruencji. Zauważmy, że już iloczyn 30 początkowych liczb pierwszych, to więcej niż $2^{128}$. 

Z tego powodu nie możemy polegać na `Int128` i musimy wykorzysytywać `BigInt` - wolniejsze, ale nieograniczone.

In [3]:
prod(PRIMES[1:30]) > BigInt(2)^128

true

# Rozwiązywanie układów kongruencji - Chińskie Twierdzenie o Resztach  

## Metoda Lagrange'a

Zaimplementuj rozwiązywanie układów kongruencji funkcję `congruence_system_lagrange(a,m)`, która przyjmuje parametry:  
1) `a` – tablica reszt modulo (dowolne liczby całkowite)  
2) `m` – tablica rozważanych modulo (parami względnie pierwsze liczby)

Funkcja powinna zwracać wartość $x$, taką, że $x \equiv a[i] \pmod{m_i}$.

Postaraj się, żeby funkcja działała dla liczb całkowitych, jak i dla wielomianów nad ciałem z paczki `Oscar.jl`

Przydatne funkcje: 
1) `gcdx(a,b)` - zwraca nwd oraz współczynniki Bezouta.
2) `div(a,b)` - zwraca dokładny iloraz liczb całkowitych. 
3) `mod(a,b)` - zwraca resztę z dzielnia $a$ przez $b$.
4) `prod(T)` - zwraca iloczyn elementów tablicy T. 

In [ ]:
function congruence_system_lagrange(a,m)
end

congruence_system_lagrange (generic function with 1 method)

### Testy poprawności działania na liczbach całkowitych.

Funkcja sprawdzająca, czy podane rozwiązanie jest prawdziwe

In [5]:
function check_solution(x, a, m)
    return all(i -> mod(x, m[i]) == mod(a[i],m[i]), 1:length(a))
end

check_solution (generic function with 1 method)

Funkcja generująca losowy układ kongruencji modulo losowe liczby pierwsze (zatem założenie o względnej pierwszości jest spełnione.)

In [6]:
function random_integer_congruence(f=100,g=1000,h=1000)
    n = rand(2:f)
    m = PRIMES[sample(1:g,n,replace=false)]
    a = BigInt.(rand(1:h,n))
    return a,m
end

random_integer_congruence (generic function with 4 methods)

Funkcja testująca poprawność działania algorytmu. Argument `solver` to wskaźnik do funkcji implementującej algorytm.

In [7]:
function test_congruence(solver)
    for i=1:100
        a,m = random_integer_congruence()
        @assert check_solution(solver(a,m),a,m) "wrong solution! $a, $m"
    end
    println("Test passed!")
end

test_congruence (generic function with 1 method)

Sprawdźmy, czy wszystko działa poprawnie:

In [8]:
test_congruence(congruence_system_lagrange)

Test passed!


## Metoda Przyrostowa Newtona

Drugi rozważany algorytm to *Metoda przyrostwa Newtona.* Pomysł opiera się na iteracyjnym zastępowaniu układem równań dwóch kongruencji jednym równaniem. W ten sposó, w każdym kroku zmniejszamy liczbę równań o jeden, aż końcowo otrzymujemy rozwiązanie.

Zaimplementuj algorytm przyrostowy Newtona. W tym celu najpierw zaimplementuj funkcję pomocniczą `congruence_two(a,m)`, która ma rozwiązywać układ dwóch kongruencji.

Paremetry:
1) `a` - dwuelementowa tablica spodziewanych reszt. 
2) `m` - dwuelementowa tablica (względnie pierwszych) rozważanych modulo.

Funkcja powinna zwracać pojedynczą wartość - "najmniejsze" rozwiązanie układu dwóch kongruencji.

In [ ]:
function congruence_two(a,m)
end

congruence_two (generic function with 1 method)

Zaimplementuj rozwiązywanie układów kongruencji funkcję jako `congruence_system_newton(m, a)`. 

Funkcja powinna korzystać z `congruence_two(m,a)`

Parametry, wartości zwracane oraz działanie ma być identyczne z `congruence_system_lagrange(m,a)`.

In [ ]:
function congruence_system_newton(a,m)
end

congruence_system_newton (generic function with 1 method)

### Testy poprawności działania na liczbach całkowitych

Sprawdźmy, czy algorytm zwraca spodziewane wyniki.

In [11]:
test_congruence(congruence_system_newton)

Test passed!


## Porównanie działania 


### Interesującym jest pytanie, która z pokazanych metod jest lepsza. 

Sprawdźmy to, generując losowe dane i mierząc czas działania obu funkcji.



In [12]:
function solve_all(solver,data)
    for (a,m) in data 
        solver(a,m)
    end
end

solve_all (generic function with 1 method)

In [13]:
n = 5000
data = [random_integer_congruence() for i=1:n]
@time solve_all(congruence_system_lagrange,data)
@time solve_all(congruence_system_newton,data)  

  7.696965 seconds (101.25 M allocations: 2.933 GiB, 24.36% gc time, 0.11% compilation time)
  0.482714 seconds (7.30 M allocations: 235.469 MiB, 20.78% gc time, 1.56% compilation time)


Polecam trochę potestować. Dla dużych wartości `n`, ewidentnie metoda przyrostowa radzi sobie znacznie lepiej.

### Testy poprawności działania na wielomianach jednej zmiennej nad ciałem $\mathbb{Q}$.

Dobrze zaimplementowany kod, powinien działać nie tylko dla liczb całkowitych, ale i dla innych pierścieni Euklidesowych - w szczególności wielomianów jednej zmiennej nad ciałem.
W celu przetestowania tego, skorzystamy z paczki `Oscar.jl` z poprzednich laboratoriów.

Nie będziemy tego testować na bardzo ogólnych przypadkach - ograniczymy się do testowania sytuacji, gdy rozważane wielomiany modularne $m_i$ są postaci $x - p_i$ dla pewnego $p_i$. Wtedy rozważane reszty $a_i$ są stopnia zerowego (znaczy są po prostu liczbami).

W takiej sytuacji, problem znalezienia wielomianu $f$, takiego, że $f \equiv a_i \pmod{m_i}$ dla każdego $i$, można równoważnie zapisać jako:
$$ \forall i \quad  f(p_i) = a_i $$
czyli jest to interpolacja wielomianu.

Zdefiniujmy pierścień $R = \mathbb{Q}[x]$

In [14]:
R, x = QQ[:x]

(Univariate polynomial ring in x over QQ, x)

Funkcja generująca losowy problem interpolacji wielomianu jako układ kongruencji.

In [15]:
function random_polynomial_congruence()
    n = 100
    a = rand(1:1000,n) 
    x_coords =  sample(1:1000,n,replace=false) 
    m = [x - x_coords[i] for i=1:n]
    return a,m
end

random_polynomial_congruence (generic function with 1 method)

Funkcja sprawdzająca poprawność działania

In [16]:
function check_solution_polynomial(f,a,m)
    k = length(m)
    x_cord = [-m[i](0) for i=1:k]
    return all(i -> f(x_cord[i]) == a[i], 1:k) 
end

check_solution_polynomial (generic function with 1 method)

In [17]:
function test_congruence_polynomial(solver)
    for i=1:100
        a,m = random_polynomial_congruence()
        @assert check_solution_polynomial(solver(a,m),a,m) "wrong solution! $a, $m"
    end
    println("Test passed!")
end

test_congruence_polynomial (generic function with 1 method)

In [150]:
test_congruence_polynomial(congruence_system_newton)

Test passed!


In [19]:
test_congruence_polynomial(congruence_system_lagrange)


Test passed!


### Prostsza implementacja rozwiązywania dwóch kongruencji.

Rozważmy układ dwóch kongruencji $x \equiv a \pmod{m_1} \land x \equiv b \pmod{m_2}$. 

Zakładamy, że $m_1,m_2$ są względnie pierwsze - zatem można wyliczyć współczynniki Bezouta: 
$$ \alpha m_1  + \beta m_2 = 1. $$

Zauważmy teraz, że rozwiązaniem układu kongruencji jest po prostu: 
$$ \alpha m_1 b + \beta m_2 a$$
Dlaczego jednak w algorytmie na prezentacji, proponowana była inna implementacja? Chodzi o wielkość pojawiających się czynników. 

Podmieńmy implementacje `congruence_two(a,m)` na taką uproszczoną:

In [20]:
function congruence_two_v2(a,m)
    _, α, β = gcdx(m[1], m[2])
    return mod(α*m[1]*a[2] + β*m[2]*a[1], m[1]*m[2])
end

congruence_two_v2 (generic function with 1 method)

Napiszmy teraz funkcję `congruence_system_newton_v2(a,m)` - kod powinien być dokładnie taki sam jak do wersji oryginalnej, tylko zamiast odwoływać się do `congruence_two` powinien odwoływać się do `congruence_two_v2`

In [ ]:
function congruence_system_newton_v2(a,m)
end

congruence_system_newton_v2 (generic function with 1 method)

Sprawdźmy teraz, czy zwraca poprawne wyniki i co ciekawsze - jak wypada wydajnościowo w kontraście do `congruence_system_newton`

In [22]:
test_congruence(congruence_system_newton_v2)

Test passed!


In [23]:
n = 10000
data = [random_integer_congruence() for i=1:n]
@time solve_all(congruence_system_newton,data)
@time solve_all(congruence_system_newton_v2,data)  

  1.105093 seconds (14.77 M allocations: 478.666 MiB, 17.15% gc time)
  1.324977 seconds (14.81 M allocations: 527.932 MiB, 22.60% gc time, 0.84% compilation time)


Wygląda na to, że wypada bardzo podobnie (polecam uruchomić pare razy - u mnie raz szybciej działa wersja podstawoa, raz ta druga.) 

Ale... podczas testowania, zauważyłem jedną rzecz. Porównajmy wydajność dla kongruencji wielomianów.

In [24]:
n = 100
data = [random_polynomial_congruence() for i=1:n]
@time solve_all(congruence_system_newton,data)
@time solve_all(congruence_system_newton_v2,data)  

  2.294649 seconds (1.82 M allocations: 278.094 MiB, 5.12% gc time, 0.40% compilation time)
  8.630788 seconds (1.73 M allocations: 2.691 GiB, 4.59% gc time, 1.35% compilation time)


W tym wypadku nowa wersja wypada **znacznie** gorzej. 

Dlaczego tak jest?

# Potęgowanie modularne

### Podstawowa metoda szybkiego potęgowania modularnego
Zaimplementuj funkcje `power_modulo(a,n,m)` która przyjmuje na wejściu liczby całkowitę `a,n,m` i zwraca wartość $a^n \pmod{m}$. Wykorzystaj do tego szybkie potęgowanie.

Przydatne funkcje:
1) `digits(n,base)` - zwraca liczbę $n$ w systemie o podstawie $base$

In [ ]:
function power_modulo(a,n,m)
end

power_modulo (generic function with 1 method)

### Testy poprawności działania potęgowania modularnego

Przetestujmy poprawność wyników. Na losowo generowanych danych, będziemy porównywać się do funkcji bibliotecznej `powermod`

In [26]:
function test_power_modulo()
    for i=1:100
        a = rand(BigInt(1):BigInt(1_000_000))
        n = rand(BigInt(1):BigInt(1_000_000))
        m = rand(BigInt(1):BigInt(1_000_000))
        @assert power_modulo(a,n,m) == powermod(a,n,m) "error for a=$a, n=$n, m=$m"
    end
    println("test passed!")
end

test_power_modulo (generic function with 1 method)

In [27]:
test_power_modulo()

test passed!


### Funkcja $\varphi$ Eulera
Zaimplementuj funkckję $\phi(n)$ przyjmującą na wejściu liczbę naturalną $n$ i zwracającą na wyjściu wartość funkcji Eulera $\varphi(n)$. 
Przydatne funkcje:
1) `factor(n)` - zwraca faktoryzację na czynniki pierwsze liczby $n$. 
Uwaga - w Paczcze `Oscar` i `Primes` osobno zdefinowano funkcje `factor()`. Żeby ominąć niejednoznaczności można pisać `Oscar.factor()` lub `Primes.factor()`. Nie ma to szczególnego znaczenia, ale jak testowałem dla bardzo dużych liczb, to `Oscar.factor` wypada lekko szybciej. 
Możemy też zapisać po prostu:

In [28]:
import Oscar.factor as factor 

In [ ]:
function ϕ(n) 
end

ϕ (generic function with 1 method)

### Testy poprawności działania

Porównajmy poprawność wyliczeń z wynikami bibliotecznej funkcji `totient`.

In [30]:
function test_phi_func()
    for i=1:1000
        n = rand(1:10^6)
        @assert ϕ(n) == totient(n)
    end
    println("Test passed!")
end

test_phi_func (generic function with 1 method)

In [31]:
test_phi_func()

Test passed!


### Szybkie Potęgowanie Modulo wykorzystując funkcje $\varphi$ Eulera.
Zaimplementuj funkcje `power_modulo_euler(a,n,m)`, która działa analogicznie do funkcji `power_modulo(n)`, wykorzystując dodatkowo twierdzenie Eulera.

In [ ]:
function power_modulo_euler(a,n,m)
end

power_modulo_euler (generic function with 1 method)

### Testy poprawności działania

Sprawdźmy wyniki, ponownie odwołując się do wyników `powermod`

In [33]:
function test_power_modulo_euler()
    for i=1:100
        a = rand(1:100000000)
        n = rand(1:100000000)
        m = rand(1:100000000)
        @assert power_modulo_euler(a,n,m) == powermod(a,n,m) "error for a=$a, n=$n, m=$m"
    end
    println("test passed!")
end

test_power_modulo_euler (generic function with 1 method)

In [34]:
test_power_modulo_euler()

test passed!


### Porównanie wydajności - ile "oszczędza" nam Twierdzenie Eulera. 

Funkcja odpowiedzialna za rozwiązywanie wielu przykładów

In [35]:
function solve_all_power(solver, data)
    for (a,n,m) in data 
        solver(a,n,m)
    end
end

solve_all_power (generic function with 1 method)

Porównamy obie metody: standardową oraz tą wykorzystującą funkcje $\varphi$ Eulera. Dodatkowo, sprawdzimy też, jak wypada funkcja biblioteczna. 

In [36]:
n = 1_000_000
data = [[rand(1:10^9), rand(1:10^9), rand(1:10^9)] for j=1:n]
@time solve_all_power(power_modulo,data)
@time solve_all_power(power_modulo_euler,data)
@time solve_all_power(powermod,data)

  0.772914 seconds (2.01 M allocations: 279.711 MiB, 19.06% gc time, 1.49% compilation time)
  4.093836 seconds (8.80 M allocations: 904.003 MiB, 5.60% gc time, 0.27% compilation time)
  0.622080 seconds (10.50 k allocations: 539.977 KiB, 1.82% compilation time)


Jak widać.... wyniki sporo poniżej oczekiwań. Liczenie funkcji Eulera jest na tyle kosztowne, że nie opłaca się tego robić, a przynajmniej nie w ten sposób. Istnieją inne metody wyznaczania jej dla kolejnych liczb naturalnych w czasie zamortyzowanie stałym. Z tego powodu, jeśli chcemy liczyć wiele razy potęge modularną, może opłacać nam sie wyliczyć i zapisać wyniki funkcji Eulera, a później z nich korzystać.

Co ciekawe natknąłem sie na taki post na [forum Julii](https://discourse.julialang.org/t/optimizing-powermod/115936)
można znaleźć tam taką implementację funkcji:

In [37]:
function my_powermod(a::Int64, b::Int64, c::Int64)
    @assert b >= 0
    result = one(Int128)
    a_raised_to_powers_of_two = convert(Int128, a)
    while b != 0
        if isodd(b)
            result *= a_raised_to_powers_of_two
            result = result % c;
        end
        b = b >>> 1
        a_raised_to_powers_of_two = a_raised_to_powers_of_two^2 % c
    end
    convert(Int64, result)
end

my_powermod (generic function with 1 method)

Zauważmy, że jest to implementacja `power_modulo()`. Po prostu, zamiast używać sztucznie `digits()` sprawdzana jest parzystość i na tej podstawie określane są kolejne bity.

Interesujące jest to, że ta funkcja jest szybsza niż funkcja wbudowana `powermod`! 

In [38]:
n = 1_000_000
data = [[rand(1:10^9), rand(1:10^9), rand(1:10^9)] for j=1:n]
@time solve_all_power(powermod,data)
@time solve_all_power(my_powermod,data)

  0.652593 seconds
  0.540188 seconds (14.94 k allocations: 762.039 KiB, 5.08% compilation time)


# Liczenie wyznacznika Macierzy całkowitoliczbowej.

Rozważmy macierz $M$ o elementach całkowitych. Jej wyznacznik $\det(M)$ należy również do liczb całkowitych. 
Klasyczne algorytmy liczenia wyznacznika, wykorzystują dzielenie, przez co implementacja algorytmu liczenia wyznacznika, może nam zwrócić niedokładną wartości z powodu nieprecyzyjnej reprezentacji liczb zmiennoprzecinkowych.

### Nie trzeba daleko szukać:

In [39]:
M = [3 1 -2;
     2 -3 -1;
    1 3 0]
println("typ elementu macierzy: $(typeof(M[1,1]))")
println("wyznacznik macierzy: $(det(M))") 

typ elementu macierzy: Int64
wyznacznik macierzy: -9.999999999999998


### Wyznacznik tej macierzy, jest równy -10. Mimo to, funkcja biblioteczna zwraca wynik niecałkowity...

Można by sobie poradzić z tym na wiele sposobów. Najprostszym pomysłem, jest reprezentacja liczb wymiernych jako ułamki (licznik/mianownik). W ten sposób moglibyśmy uzyskać dokładny wynik, ale działanie na ułamkach w ten sposób jest drogie - mianowniki zwykle bardzo szybko rosną. 

Sprytniejszym pomysłem jest zastosowanie *chińskiego twierdzenia o resztach*. 
Jeżeli tylko ograniczymy wartośc wyznacznika, można policzyć wyznacznik modulo wiele liczb pierwszych, a następnie złączyć wyniki modulo małe liczby pierwsze do modulo na tyle dużego - że otrzymamy jednoznaczny wynik.

### Nierówność Hadamarda 

Ograniczenie górne na wartość wyznacznika znane jest jako nierówność Hadamarda. To ją wykorzystamy. 

Zaimplementuj funkcję `hadamard_bounding(M)`, która przyjmuję na wejściu macierz $M$ i zwraca ograniczenie górne (całkowitoliczbowe) na jej wyznacznik.

In [ ]:
function hadamard_bounding(M)
end

hadamard_bounding (generic function with 1 method)

### Testy poprawności działania 

(na bardzo pojedynczych przykładach)

In [41]:
function test_hadamard()
    M1 = [3 1 -2; 2 -3 -1;1 3 0]
    M2 = reshape(1:16, 4, 4)
    M3 = [87  42  73  10; 25  11  99  58; 67  30  14  77; 92  51  60  18 ]
    @assert hadamard_bounding(M1) == 141 "error in M1"
    @assert hadamard_bounding(M2) == 1048576 "error in M2"
    @assert hadamard_bounding(M3) == 1536953616 "error in M3"
    println("Test passed!")
end

test_hadamard (generic function with 1 method)

In [42]:
test_hadamard()

Test passed!


### Odwrotność elementu w ciele $\mathbb{Z}_p$ 
Zaimplementuj funkcje `my_inverse(a,p)`, która przyjmuje zwraca odwrotność liczby $a$ względem mnożenia w ciele $\mathbb{Z}_p$.

In [ ]:
function my_inverse(a,p)
end

my_inverse (generic function with 1 method)

### Eliminacja Gaussa Modulo p
Zaimplementuj funkcję `gauss_modulo(M,n,p)`, która liczy wyznacznik macierzy $M$ o rozmiarze $n \times n$ modulo $p$.

In [ ]:
function gauss_modulo(M,n,p)
end

gauss_modulo (generic function with 1 method)

### Dobór liczb pierwszych


Zaimplementuj funkcje `get_primes(BOUND)`, która zwraca tablicę liczb pierwszych, których iloczyn jest większy niż `BOUND`. Dodatkowo, powinna zwracać wartość tego iloczynu. 

Narazie rozważmy najprostszą implementacje - skorzystajmy z tablicy `PRIMES`. Bierzmy kolejne liczby pierwsze od początku, aż ich iloczyn nie będzie odpowiednio duży.

In [ ]:
function get_primes(BOUND)
end

get_primes (generic function with 1 method)

### Liczenie wyznacznika Macierzy Całkowitoliczbowej.

Zaimplementuj funkcję `modular_determinant(M)` która liczy wyznacznik macierzy całkowitoliczbowej $M$ wykorzystując obliczenia modulo wiele liczb pierwszych $p$.


In [ ]:
function modular_determinant(M)
end

modular_determinant (generic function with 1 method)

### Testy poprawności działania

Sprawdźmy na losowo generowanych danych, poprawność wynikow zwracanych przez `modular_determinant`

In [47]:
function test_modulo_determinant()
    for i=1:50
        n = rand(1:30)
        M = rand(BigInt(-10):BigInt(10),n,n)
        x = modular_determinant(M)
        y = det(M) 
        @assert isapprox(x,y) "$x, $y"
    end
    println("Test passed!")
end

test_modulo_determinant (generic function with 1 method)

In [48]:
test_modulo_determinant()

Test passed!


### Testy wydajności

W celu zmierzenia wydajnośći, będziemy porównywać się do zaimplementowanej tu funkcji `gauss_determinant(M)`. 
Funkcja ta, realizuje wspomniane wcześniej prostsze podejście pozwalające na implementację dokładnego algorytmu liczenia wyznacznika.
Julia posiada typ liczby wymiernej - `Rational`, który operuje na liczbach wymiernych bez błędów przybliżeń. 
Poniższy algorytm to po prostu liczenie wyznacznika eliminacją gaussa ale zapisując dzielenie na liczbach wymiernych w sposób dokładny. 

In [49]:
function gauss_determinant(M)
    n = size(M,1)
    A = Rational{BigInt}.(copy(M))
    det = 1.0  

    for k in 1:n-1
        max_row = k
        max_val = abs(A[k, k])
        for i in k+1:n
            if abs(A[i, k]) > max_val
                max_val = abs(A[i, k])
                max_row = i
            end
        end

        if max_row != k
            A[[k, max_row], :] = A[[max_row, k], :]
            det *= -1
        end

        if A[k, k] == 0
            return 0.0
        end

        for i in k+1:n
            factor = A[i, k] / A[k, k]
            A[i, k:end] .-= factor * A[k, k:end]
        end
    end

    for i in 1:n
        det *= A[i, i]
    end

    return det
end


gauss_determinant (generic function with 1 method)

Porównajmy wyniki dla losowej macierzy 100 x 100

In [50]:
n = 100
M = rand(BigInt(-10000):BigInt(10000),n,n)

@time  a = modular_determinant(M)
@time  b = gauss_determinant(M)
@assert isapprox(a,b) 

 34.701215 seconds (546.06 M allocations: 13.533 GiB, 28.19% gc time)
  2.650054 seconds (7.02 M allocations: 347.105 MiB, 14.11% gc time, 17.63% compilation time)


Jak widać pierwsze wnioski jakie można wyciągnąć, to że ten pomysł jest zupełnie nieopłacalny.

Sprawdźmy z ciekawości, jak jeszcze radzi sobie funkcja wbudowana

In [51]:
@time det(M)

  0.245862 seconds (3.59 M allocations: 171.782 MiB, 1.58% compilation time)


1861502978808209722133763105337800511553052634997569170147661521890334067595534119200260979039270601833440599529103236168008889527426215743984748226094515981743452477814011904582089705372300864655037770058511883675920049679872823416094218044593626055040883878679853867705903079708558842415182703884275931237603346070123720919305484141621367758125722101769895942255528646345134213361168806848289438003152722284503219325886546211312867546158673402800066798

### Zmiana sposobu doboru liczb pierwszych.

Obecnie funkcja `get_primes` rozważa kolejne liczby pierwsze od początku - to są bardzo małe wartości. Dlatego bierzemy ich bardzo dużo i całość jest nieopłacalna. 

Okazuje się jednak, że operacje modulo na ogromnych liczbach, wcale nie są wiele wolniejsze. Dzięki temu, możemy znacznie zmniejszyć liczbę liczonych osobno modulo.

Najpierw wygenerujmy pierwsze 1000 liczb pierwszych większych od $2^{500}$.


In [52]:
function get_big_primes(k=1000,exp=500)
    BIGPRIMES = BigInt[]
    p = nextprime(BigInt(2)^exp)
    for i=1:k 
        push!(BIGPRIMES,p)
        p = nextprime(p+1)
    end 
    return BIGPRIMES
end

get_big_primes (generic function with 3 methods)

In [53]:
BIGPRIMES = get_big_primes()

1000-element Vector{BigInt}:
 3273390607896141870013189696827599152216642046043064789483291368096133796404674554883270092325904157150886684127560071009217256545885393053328527589431
 3273390607896141870013189696827599152216642046043064789483291368096133796404674554883270092325904157150886684127560071009217256545885393053328527589511
 3273390607896141870013189696827599152216642046043064789483291368096133796404674554883270092325904157150886684127560071009217256545885393053328527589697
 3273390607896141870013189696827599152216642046043064789483291368096133796404674554883270092325904157150886684127560071009217256545885393053328527589929
 3273390607896141870013189696827599152216642046043064789483291368096133796404674554883270092325904157150886684127560071009217256545885393053328527589973
 3273390607896141870013189696827599152216642046043064789483291368096133796404674554883270092325904157150886684127560071009217256545885393053328527590451
 3273390607896141870013189696827599152216642046043064

Teraz, zmodyfikujmy funkcję `get_primes` z której korzysta funkcja `modular_determinant`. Będziemy po prostu brać kolejne liczby pierwsze, ale z listy `BIGPRIMES`.

In [54]:
function get_primes(BOUND)
    P = 1
    i = 0
    while P <= BOUND
        i+=1
        P*= BIGPRIMES[i]
    end
    return BIGPRIMES[1:i],P
end

get_primes (generic function with 1 method)

Spróbujmy teraz porównać czasy działania

In [55]:
n = 50
M = rand(BigInt(-100000):BigInt(100000),n,n)

@time  a = modular_determinant(M)
@time  b = gauss_determinant(M)
@time det(M)

@assert isapprox(a,b) 

  0.065388 seconds (845.42 k allocations: 43.387 MiB, 33.28% compilation time: 100% of which was recompilation)
  0.114428 seconds (775.83 k allocations: 33.548 MiB)
  0.016530 seconds (440.47 k allocations: 16.050 MiB)


Jak widać, to dużo zmieniło. Juz dla $n=50$ widać nasz algorytm radzi sobie sporo lepiej od naiwnego.


Coraz większe różnicę widać dla coraz większych macierzy. Ogólnie ten algorytm ma zastosowanie dla naprawdę dużych danych.


Spróbujmy policzyć dla jeszcze trochę większego przykładu (Uwaga - u mnie to trwało ok.7 minut.)

In [97]:
n = 300
M = rand(BigInt(-10^6):BigInt(10^6),n,n)

@time  a = modular_determinant(M)
@time  b = gauss_determinant(M)
@time det(M)

@assert isapprox(a,b) 

136.984036 seconds (1.24 G allocations: 64.303 GiB, 23.05% gc time)
248.020029 seconds (163.12 M allocations: 15.992 GiB, 2.16% gc time)
 33.842999 seconds (98.42 M allocations: 13.933 GiB, 9.83% gc time)


Przy okazji, zauważmy o jakiej skali liczb mówimy. Oczywiście, wszystko zależy od tego, jaka macierz zostanie wylosowana, ale poniższe wyrażenie w moim przypadku zwraca prawdę.

In [103]:
abs(a) > BigInt(2)^5000

true

### Implementacja na Ciałach Skończonych

W ramach zabawy, polecam spróbować przepisać implementację `gauss_modulo` oraz `modular_determinant` na takie, żeby bezpośrednio korzystały z implementacji ciał skończonych z paczki `Oscar`. 

Dzięki temu, zapis samych operacji staje się trochę prostszy, redukcje modulo dzieją się pod spodem, a nie musimy ich bezpośrednio zapisywać. 

Warto zaznaczyć, że takie podejście jest raczej wolniejsze. 

Poniżej kilka linijek kodu, które pokażą podstawowe operacje na elementach ciała skończonego. Jeżeli potrzeba więcej informacji, zachęcam do przeszukiwania dokumentacji.

In [ ]:
# Stworzenie ciała skończonego 7 elementowego (izomorficznego z Z_7)
F = GF(7)

# Zrzutowanie liczb całkowitych na elementy ciała 
a = F(12)
b = F(3)
println("a = $a, b = $b")
println("a + b = $(a+b)")

# Zrzutowanie całej macierzy:
A = [-5 1 2; 3 4 10; 9 -100 4]
A2 = matrix(F,A)
println("A2 = $(A2)")

#"podniesienie" elementu z ciala na element z liczb calkowitych: (prawdopodobnie implementacja congruence_system_newton/lagrange nie bedzie działać na elemnetach z ciała ale na ZZelem już tak)
println("a = $a, typ a: $(typeof(a))")
c = lift(ZZ,a)
println("c = $c, typ c: $(typeof(c))")

a = 5, b = 3
a + b = 1
A2 = [2 1 2; 3 4 3; 2 5 4]
a = 5, typ a: FqFieldElem
c = 5, typ c: ZZRingElem


UWAGA! na elementach ciała skończonego nie ma zdefiniowanego porządku - przez to nie działa porównywanie po wartościach. Można się zastanowić dlaczego jest tak, a nie inaczej.

In [65]:
a > b

MethodError: MethodError: no method matching isless(::FqFieldElem, ::FqFieldElem)
The function `isless` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  isless(!Matched::Missing, ::Any)
   @ Base missing.jl:87
  isless(::Any, !Matched::Missing)
   @ Base missing.jl:88
  isless(!Matched::EmbeddedNumFieldElem, ::Any)
   @ Hecke ~/.julia/packages/Hecke/fZhGW/src/NumField/Embedded.jl:144
  ...


Funkcja `gauss_finite_field(A)` przyjmuje macierz $A$ i zwraca jej wyznacznik liczony metodą gaussa. 

Z założenia macierz $A$ powinna być macierzą o elementach z ciała skończonego 

In [ ]:
function gauss_finite_field(A)
end

gauss_finite_field (generic function with 1 method)

Funkcja `modular_determinant_finite_field(M)` przyjmuje macierz całkowitoliczbową $M$ i zwraca jej wyznacznik rzutując wielokrotnie macierz $M$ na macierze z ciał skończonych.

In [ ]:
function modular_determinant_finite_field(M)
end

modular_determinant_finite_field (generic function with 1 method)

### Testy poprawności działania
Sprawdźmy na losowych danych, zgodność działania z funkcją biblioteczną.

In [144]:
function test_modulo_determinant_finite_field()
    for i=1:30
        n = rand(1:30)
        M = rand(BigInt(-100):BigInt(100),n,n)
        x = BigInt(modular_determinant_finite_field(M))
        y = det(M) 
        @assert isapprox(x,y) "$x, $y"
    end
    println("Test passed!")
end

test_modulo_determinant_finite_field (generic function with 1 method)

In [145]:
test_modulo_determinant_finite_field()

Test passed!


# Zadanie Dodatkowe - szybkie mnożenie wielomianów o współczynnikach całkowitych za pomocą szybkiej transformacji Fouriera.

Zauważmy, że pomysł, który stosujemy przy liczeniu wyznacznika macierzy całkowitoliczbowej, jest bardzo łatwo rozszerzalny na inne problemy. O ile tylko potrafimy znaleźć ograniczenie górne naszego wyniku oraz nie ma znaczenia, czy najpierw dokonamy redukcji modulo, a później policzymy naszą funkcję, czy na odwrót, to jesteśmy w stanie zastosować analogiczny mechanizm. 

Rozważmy dwa wielomiany $f,g  \in \mathbb{Z}[x]$. W oczywisty sposób $f \cdot g \in \mathbb{Z}$. 

Klasyczne mnożenie wielomianów, jak "mnożenie w słupku" ma złożoność kwadratową.

Okazuje się jednak, że wykorzystując algorytm szybkiej transformacji Fouriera, można zredukować tę złożoność do liniowo-logarytmicznej.

Niestety  tracimy wtedy na precyzji - klasyczna implementacja sprawia, że natrafiamy na niedokładnie reprezentowane liczby zmiennoprzecinkowe i wynik mnożenia dwóch wielomianów o współczynnikach całkowitych posiada niecałkowite czynniki.

Ponownie, można się pozbyć tego problemu wykorzystując operacje na elementach z ciał skończonych.

Przykładowe materialy o mnożeniu wielomianów za pomocą fft:
- [Eric Bannatyne - CS Toronto Algorithm Design, Analysis & Complexit notes, 2017](https://www.cs.toronto.edu/~denisp/csc373/docs/tutorial3-adv-writeup.pdf)
- [Geeks for Geeks - Fast Fourier Transformation for polynomial multiplication, 2023](https://www.geeksforgeeks.org/fast-fourier-transformation-poynomial-multiplication/), 
- [Przemysław Koprowski -  Wykład z Matematyki , 20202](https://www.youtube.com/watch?v=A_qRvhf_z-E), 
- [Michał Dobranowski - Notatki, 2022](https://mdbrnowski.github.io/notes/pdf/fft.pdf)

Dla jasności - żadnych z tych materiałów nie czytałem.